### Modeling of manually-derived strains
Computer-aided cell factory engineering was quite recently introduced to metabolic engineering, as a high throuput analysis tool to identify gene modifications and simulate the chaotic metabolic processes incide the cells. However, metabolic engineering has a longer history, and there have been a lot of efforts to achieve high production of desired products, by lowering or increasing the expression of intuitive and logically derived gene targets.
In this section, we are going to evaluate some popular manual derived changes through model simulations. Those are the downregulation of squalene synthase (<i>ERG9</i>), squalene epoxidase (<i>ERG1</i>), and sterol C-5 desaturase (<i>ERG3</i>) the upregulation of 3-hydroxy-3-methylglutaryl-coenzyme A reductase (<i>HMG1</i>) and the deletion of aldehyde dehydrogenase (<i>ALD6</i>).

Loading the model

In [72]:
from cobra.io import read_sbml_model, write_sbml_model
model = read_sbml_model('/Users/nikow/github-classroom/27410/group-assingment-team-4/yeast-GEM-8.7.0/model/yeast-GEM3.xml') # loading
from cobra import Reaction, Metabolite, Gene
import pandas as pd

In [102]:
model.optimize

AttributeError: 'Model' object has no attribute 'objective_value'

We will locate the reactions in the model

In [73]:
model.reactions.query('squalene synthase', 'name')

[<Reaction r_1012 at 0x29ee207ab10>]

In [74]:
model.reactions.r_1012

Reaction identifier,r_1012
Name,squalene synthase
Memory address,0x29ee207ab10
Stoichiometry,2.0 s_0190 + s_0794 + s_1212 --> 2.0 s_0633 + s_1207 + s_1447 2.0 farnesyl diphosphate + H+ + NADPH --> 2.0 diphosphate + NADP(+) + squalene
GPR,YHR190W
Lower bound,0.0
Upper bound,1000.0


In [75]:
model.reactions.query('squalene epoxidase', 'name')

[<Reaction r_1010 at 0x29ee20e3050>, <Reaction r_1011 at 0x29ee20e3bd0>]

In [76]:
model.reactions.r_1010

Reaction identifier,r_1010
Name,squalene epoxidase (NAD)
Memory address,0x29ee20e3050
Stoichiometry,"s_0795 + s_1204 + s_1276 + s_1448 --> s_0038 + s_0804 + s_1199 H+ + NADH + oxygen + squalene --> (S)-2,3-epoxysqualene + H2O + NAD"
GPR,YGR175C
Lower bound,0.0
Upper bound,1000.0


In [77]:
model.reactions.query('sterol desaturase', 'name')

[<Reaction r_0233 at 0x29ee1e13b10>, <Reaction r_0242 at 0x29ee1e34a10>]

In [78]:
model.reactions.r_0242

Reaction identifier,r_0242
Name,C-5 sterol desaturase
Memory address,0x29ee1e34a10
Stoichiometry,"s_0657 + s_0794 + s_1212 + s_1275 --> s_0664 + 2.0 s_0803 + s_1207 episterol + H+ + NADPH + oxygen --> ergosta-5,7,24(28)-trien-3beta-ol + 2.0 H2O + NADP(+)"
GPR,YLR056W
Lower bound,0.0
Upper bound,1000.0


In [79]:
model.reactions.query('hydroxymethylglutaryl CoA reductase', 'name')

[<Reaction r_0558 at 0x29ee1f53890>]

In [80]:
model.reactions.r_0558

Reaction identifier,r_0558
Name,hydroxymethylglutaryl CoA reductase
Memory address,0x29ee1f53890
Stoichiometry,s_0218 + 2.0 s_0794 + 2.0 s_1212 --> s_0028 + s_0529 + 2.0 s_1207 3-hydroxy-3-methylglutaryl-CoA + 2.0 H+ + 2.0 NADPH --> (R)-mevalonate + coenzyme A + 2.0 NADP(+)
GPR,YLR450W or YML075C
Lower bound,0.0
Upper bound,1000.0


<i>S. cerevisiae</i> posseses two genes encoding 3-hydroxy-3-methylglutaryl-coenzyme A reductase enzymes, <i>HMG1</i> and <i>HMG2</i>. As the isoenzyme HMG1 appears to be the more stable version of the two, it is the one usually picked for upregulation.

In [81]:
model.reactions.query('YPL061W' , 'gene_name_reaction_rule')

[<Reaction r_0173 at 0x29ee1dbfe10>,
 <Reaction r_0177 at 0x29ee1dc3850>,
 <Reaction r_4718 at 0x29ee331ab10>,
 <Reaction r_4719 at 0x29ee331b3d0>]

In [82]:
model.reactions.r_0173

Reaction identifier,r_0173
Name,"aldehyde dehydrogenase (acetaldehyde, NADP)"
Memory address,0x29ee1dbfe10
Stoichiometry,s_0359 + s_0803 + s_1207 --> s_0362 + 2.0 s_0794 + s_1212 acetaldehyde + H2O + NADP(+) --> acetate + 2.0 H+ + NADPH
GPR,YPL061W
Lower bound,0.0
Upper bound,1000.0


In [83]:
model.reactions.r_0177

Reaction identifier,r_0177
Name,"aldehyde dehydrogenase (indole-3-acetaldehyde, NADP)"
Memory address,0x29ee1dc3850
Stoichiometry,s_0803 + s_0850 + s_1207 --> 2.0 s_0794 + s_0853 + s_1212 H2O + indol-3-ylacetaldehyde + NADP(+) --> 2.0 H+ + indole-3-acetate + NADPH
GPR,YPL061W
Lower bound,0.0
Upper bound,1000.0


In [84]:
model.reactions.r_4718

Reaction identifier,r_4718
Name,"isobutyraldehyde dehydrogenase, c"
Memory address,0x29ee331ab10
Stoichiometry,s_0803 + s_0937 + s_1198 <=> 2.0 s_0794 + s_1203 + s_4285 H2O + isobutyraldehyde + NAD <=> 2.0 H+ + NADH + isobutyrate
GPR,YOR374W or YPL061W
Lower bound,-1000.0
Upper bound,1000.0


In [85]:
model.reactions.r_4719

Reaction identifier,r_4719
Name,"2-methylbutanal dehydrogenase, c"
Memory address,0x29ee331b3d0
Stoichiometry,s_0166 + s_0803 + s_1198 <=> 2.0 s_0794 + s_1203 + s_4282 2-methylbutanal + H2O + NAD <=> 2.0 H+ + NADH + 2-methylbutyrate
GPR,YOR374W or YPL061W
Lower bound,-1000.0
Upper bound,1000.0


In [86]:
model.reactions.query('YOR374W' , 'gene_name_reaction_rule')

[<Reaction r_0174 at 0x29ee1dc87d0>,
 <Reaction r_0175 at 0x29ee1dce490>,
 <Reaction r_0176 at 0x29ee1d8d050>,
 <Reaction r_0178 at 0x29ee1d7b5d0>,
 <Reaction r_0201 at 0x29ee1dd7610>,
 <Reaction r_2116 at 0x29ee23afd90>,
 <Reaction r_4678 at 0x29ee32e3d10>,
 <Reaction r_4718 at 0x29ee331ab10>,
 <Reaction r_4719 at 0x29ee331b3d0>]

It appears that ALD6 catalyzes 4 reactions in total, for two of them being the only enzyme doing so. For the later two, another aldehyde dehydrogenase, ALD4, has the abillity to catalyze the same reactions. As a conclusion, to demosntrate the deletion of <i>ALD6</i>, we will delete reactions r_0173 and r_0177, as they are the main reactions our enzyme is involved into.

For the next step, we will simulate the modifications in our model.

<i>ALD6</i> deletion

In [87]:
model.reactions.r_0173

Reaction identifier,r_0173
Name,"aldehyde dehydrogenase (acetaldehyde, NADP)"
Memory address,0x29ee1dbfe10
Stoichiometry,s_0359 + s_0803 + s_1207 --> s_0362 + 2.0 s_0794 + s_1212 acetaldehyde + H2O + NADP(+) --> acetate + 2.0 H+ + NADPH
GPR,YPL061W
Lower bound,0.0
Upper bound,1000.0


In [88]:
model.reactions.r_0177

Reaction identifier,r_0177
Name,"aldehyde dehydrogenase (indole-3-acetaldehyde, NADP)"
Memory address,0x29ee1dc3850
Stoichiometry,s_0803 + s_0850 + s_1207 --> 2.0 s_0794 + s_0853 + s_1212 H2O + indol-3-ylacetaldehyde + NADP(+) --> 2.0 H+ + indole-3-acetate + NADPH
GPR,YPL061W
Lower bound,0.0
Upper bound,1000.0


In [96]:
with model:
    model.reactions.r_0173.knock_out()
    model.reactions.r_0177.knock_out()
    
    growth_rate = model.optimize().objective_value
    print('Growth rate after deletion:', growth_rate, 'gDW h^-1')
    
    

Growth rate after deletion: 20.353598390187724 gDW h^-1


In [100]:
with model:
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")


Maximum beta-farnesene productivity =  20.44590733027539 mmol/gDW*h
Maximum theoretical yield is =  1.0222953665137695 mmol/gDW*h


<i>HMG1</i> upregulation

We are going to simulate the upregulation of <i>HMG1</i> by increasing the lower bound of the HMGR reaction in our model to 0.1 from 0.0.


In [103]:
with model:
    model.reactions.r_0558.lower_bound = 0.1
    
    growth_rate = model.optimize().objective_value
    print('Growth rate after upregulation:', growth_rate, 'gDW h^-1')
    
    

Growth rate after upregulation: 20.445907330275297 gDW h^-1


In [105]:
with model:
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")


Maximum beta-farnesene productivity =  20.445907330275297 mmol/gDW*h
Maximum theoretical yield is =  1.0222953665137648 mmol/gDW*h


<i>ERG9</i> downregulation

Next, we are going to downregulate the expression of <i>ERG9</i> in our model, by decreasing the upper bound of the reaction. Remember that deletion of that gene is lethal, so we have to be careful with how much we are decrease the upper bound. After consideration, we choose the value of 0.1.

In [91]:
model.reactions.r_1012

Reaction identifier,r_1012
Name,squalene synthase
Memory address,0x29ee207ab10
Stoichiometry,2.0 s_0190 + s_0794 + s_1212 --> 2.0 s_0633 + s_1207 + s_1447 2.0 farnesyl diphosphate + H+ + NADPH --> 2.0 diphosphate + NADP(+) + squalene
GPR,YHR190W
Lower bound,0.0
Upper bound,1000.0


In [92]:
with model:
    model.reactions.r_1012.upper_bound = 0.1
    
    growth_rate = model.optimize().objective_value
    print('Growth rate after downregulation:', growth_rate, 'gDW h^-1')
    
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")

Growth rate after downregulation: 20.445907330275404 gDW h^-1
Maximum beta-farnesene productivity =  20.445907330275404 mmol/gDW*h
Maximum theoretical yield is =  1.0222953665137702 mmol/gDW*h


In the same way as <i>ERG9</i> we are going to model the downregulation of <i>ERG1</i> and <i>ERG3</i>.

Downregulation of <i>ERG1</i>

In [93]:
with model:
    model.reactions.r_1010.upper_bound = 0.1
    
    growth_rate = model.optimize().objective_value
    print('Growth rate after downregulation:', growth_rate, 'gDW h^-1')
    
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")

Growth rate after downregulation: 20.445907330275404 gDW h^-1
Maximum beta-farnesene productivity =  20.445907330275404 mmol/gDW*h
Maximum theoretical yield is =  1.0222953665137702 mmol/gDW*h


Downregulation of <i>ERG3</i>

In [94]:
with model:
    model.reactions.r_0242.upper_bound = 0.1
    
    growth_rate = model.optimize().objective_value
    print('Growth rate after downregulation:', growth_rate, 'gDW h^-1')
    
    model.objective = model.reactions.BFD

    beta_farnesene_max_production = model.optimize().objective_value
    print("Maximum beta-farnesene productivity = ",beta_farnesene_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.r_1714.flux #Glucose exchange reaction
    max_yield = beta_farnesene_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")

Growth rate after downregulation: 20.445907330275404 gDW h^-1
Maximum beta-farnesene productivity =  20.445907330275404 mmol/gDW*h
Maximum theoretical yield is =  1.0222953665137702 mmol/gDW*h


### Conclusions
We simulated some frequently used in literature manually derived metabolic modifications in <i>S. cerevisiae</i> in order to increase terpenoid (and beta-farnesene in our case) production. 
Our analysis showed that the deletion of <i>ALD6</i> is 


In [95]:
model.medium

{'r_1654': 1000.0,
 'r_1714': 20.0,
 'r_1810': 10.0,
 'r_1832': 1000.0,
 'r_1861': 10.0,
 'r_1873': 10.0,
 'r_1879': 10.0,
 'r_1881': 10.0,
 'r_1883': 10.0,
 'r_1891': 10.0,
 'r_1893': 10.0,
 'r_1897': 10.0,
 'r_1899': 10.0,
 'r_1900': 10.0,
 'r_1902': 10.0,
 'r_1903': 10.0,
 'r_1904': 10.0,
 'r_1906': 10.0,
 'r_1911': 10.0,
 'r_1912': 10.0,
 'r_1913': 10.0,
 'r_1914': 10.0,
 'r_1992': 1000.0,
 'r_2005': 1000.0,
 'r_2020': 1000.0,
 'r_2049': 1000.0,
 'r_2060': 1000.0,
 'r_2100': 1000.0,
 'r_4593': 1000.0,
 'r_4594': 1000.0,
 'r_4595': 1000.0,
 'r_4596': 10.0,
 'r_4597': 1000.0,
 'r_4600': 1000.0}